In [4]:
import requests
import bs4
import os
import pandas as pd
import json
import numpy as np 
from unidecode import unidecode
import re
import pdb

# Problematique : QUELS SONT LES ALIMENTS LES PLUS MAUVAIS POUR LA SANTE ? = LES PLUS GRAS 
# Quelles sont les enseignes qui vendent les produits les plus gras ? 

In [42]:
aliments = pd.read_csv('aliments.csv', sep='\t')
#aliments = aliments.set_index('product_name')
#aliments.dropna(subset=['fat_100g'])['fat_100g'].str.split(',', expand=True)
#aliments.dropna(subset=['fat_100g'])['fat_100g'].str.split(',', expand=True).to_csv('traces.csv')

#aliments_with_traces = aliments.dropna(subset=['fat_100g'])

In [43]:
aliments.head()


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
0,1159,http://world-fr.openfoodfacts.org/produit/1159,NaN,1350389575,2012-10-16T12:12:55Z,1350389575,2012-10-16T12:12:55Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2103,http://world-fr.openfoodfacts.org/produit/2103,NaN,1350411133,2012-10-16T18:12:13Z,1350411133,2012-10-16T18:12:13Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2885,http://world-fr.openfoodfacts.org/produit/2885,NaN,1361357626,2013-02-20T10:53:46Z,1361357626,2013-02-20T10:53:46Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22555,http://world-fr.openfoodfacts.org/produit/22555,NaN,1353254187,2012-11-18T15:56:27Z,1353254187,2012-11-18T15:56:27Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,150127,http://world-fr.openfoodfacts.org/produit/150127,NaN,1362830041,2013-03-09T11:54:01Z,1362830041,2013-03-09T11:54:01Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [263]:
# Analyse groupee, filtres

# on prend les plus gras
gras = aliments.groupby(aliments['product_name'])['fat_100g'].mean()
produits_fat = aliments['product_name']
gras = gras.dropna()
produits_fat = produits_fat.dropna()
df = pd.concat([gras, produits_fat], axis=1)
print produits_fat

aliments['fat_100g'].describe()

28                                 Cod liver oil (softgel)
30                                             Gouda vieux
33                   Thé de Noël aromatisé orange-cannelle
34                                                   Flute
35                                            Zumo de Piña
36                                             Cauliflower
37                                           Spring Onions
38                                   Pepsi, Nouveau goût !
39                      Trader Joe's sea salt potato chips
40                  Marks & Spencer Curiously Strong Mints
41                                    Large flat mushrooms
42                                      Velouté de légumes
43                                                Baguette
44                                        Rollmop Herrings
45                                      Creamy Chicken Pie
47                                                Broccoli
48                                Barres Goût Miel Amand

count    14265.000000
mean        12.545009
std         18.120572
min          0.000000
25%               NaN
50%               NaN
75%               NaN
max        731.000000
Name: fat_100g, dtype: float64

In [262]:
# on trie les aliments par les plus gras et on retire les mots "huile", "beurre", "margarine" avec contains

fat2 = aliments[aliments['fat_100g']>85]
fat = DataFrame(pd.concat([fat2['fat_100g'], fat2['product_name'], fat2['brands']], axis=1))

fat[['fat_100g']] = fat[['fat_100g']].apply(pd.to_numeric)
fat.reset_index(inplace=True)
del(fat['index'])
fat.dropna()

# on supprime les huiles
huiles = []
for i in range(len(fat)) : 
    #if str(fat.ix[i]['product_name']).find('Huile' or 'Aceite' or 'huile') == -1 :
    if str(fat.ix[i]['product_name']).find('Huile') != -1 :
        huiles.append(True)
    elif str(fat.ix[i]['product_name']).find('Aceite') != -1 :
        huiles.append(True)
    elif str(fat.ix[i]['product_name']).find('huile') != -1 :
        huiles.append(True)
    else :
        huiles.append(False)
#print huiles
fat['H'] = DataFrame(huiles)
#print fat.head()
fat2 = fat[fat['H']==False]


# on trie par ordre croissant et on ne garde que les 50 derniers produits

#sorted(fat2)
fat3 = DataFrame(fat2.sort_values(['fat_100g'], ascending=[False]))
del(fat3['H'])
print fat3[:10]


     fat_100g                                       product_name  \
0       731.0   Kiri® Goûter - 280 g (8 barquettes) - TEST IMAGE   
65      200.0                             t budget compote pomme   
44      100.0                     Lesieur Secret de méditerranée   
146     100.0                                 Azeite tradicional   
142     100.0                                     Óleo alimentar   
135     100.0  Mermelada de manzana y nueces &quot;Esencia An...   
94      100.0                                          Santoliva   
88      100.0                               Finest Beef Dripping   
47      100.0                                Isio 4 Mini Lesieur   
78      100.0                                     spéciale pizza   

                                                brands  
0                                                  NaN  
65                                                 NaN  
44                                             Lesieur  
146                    

In [278]:
# Quelles marques vendent les produits les plus gras ? 

brands_gras = DataFrame(aliments_gras.groupby(aliments_gras['brands'])['fat_100g'].mean())
brands_gras[['fat_100g']] = brands_gras[['fat_100g']].apply(pd.to_numeric)
brands_gras.dropna()
B1 = brands_gras.sort_values(['fat_100g'],ascending=[False])
print B1

                                                    fat_100g
brands                                                      
Lesieur                                                100.0
Echinac                                                100.0
Ensemble,Biocoop                                       100.0
Esencia Andalusí, //Propiedad de://, José Serra...     100.0
Esencia Andalusí,//Propiedad de://,José Serrano...     100.0
Hacendado, //Propiedad de://, Mercadona S.A.           100.0
Hojiblanca,//Propiedad de://,Deoleo S.A.               100.0
Koipe,//Propiedad de://,Koipe Corporación S.L.,...     100.0
LESIEUR                                                100.0
Lesieur,Isio 4                                         100.0
MONINI                                                 100.0
Monini,Lesieur                                         100.0
Oleoestepa,//Propiedad de://,Oleoestepa S.C.A.         100.0
Oro de Génave,//Propiedad de://,Sierra de Génav...     100.0
Paliga                  